# Guilded Project: Designing and Creating a Database

## Exploratory data analysis:

In [2]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [4]:
game_log = pd.read_csv('game_log.csv')
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(game_log.shape)
game_log.head(10)

(171907, 161)


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [6]:
game_log.tail(10)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171897,20161002,0,Sun,OAK,AL,162,SEA,AL,162,3,2,54.0,D,NaN,NaN,NaN,SEA03,24856.0,154.0,102000000,000020000,31.0,7.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0,4.0,0.0,9.0,0.0,0.0,3.0,0.0,5.0,4.0,2.0,2.0,0.0,0.0,27.0,9.0,0.0,0.0,1.0,0.0,32.0,6.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,1.0,0.0,4.0,5.0,3.0,3.0,0.0,0.0,27.0,8.0,0.0,0.0,3.0,0.0,joycj901,Jim Joyce,fairc901,Chad Fairchild,hoyej901,James Hoye,hudsm901,Marvin Hudson,NaN,NaN,NaN,NaN,melvb001,Bob Melvin,servs002,Scott Servais,manas001,Sean Manaea,hernf002,Felix Hernandez,axfoj001,John Axford,vogts001,Stephen Vogt,manas001,Sean Manaea,hernf002,Felix Hernandez,semim001,Marcus Semien,6.0,wendj002,Joey Wendle,4.0,vogts001,Stephen Vogt,10.0,healr001,Ryon Healy,5.0,alony001,Yonder Alonso,3.0,maxwb001,Bruce Maxwell,2.0,eibnb001,Brett Eibner,8.0,olsom001,Matt Olson,9.0,smolj002,Jake Smolinski,7.0,hereg002,Guillermo Heredia,8.0,omals001,Shawn O'Malley,6.0,canor001,Robinson Cano,10.0,gutif001,Franklin Gutierrez,9.0,seagk001,Kyle Seager,5.0,lee-d004,Dae-Ho Lee,3.0,gameb001,Ben Gamel,7.0,iannc001,Chris Iannetta,2.0,freem002,Mike Freeman,4.0,NaN,Y
171898,20161002,0,Sun,TBA,AL,162,TEX,AL,162,6,4,60.0,D,NaN,NaN,NaN,ARL02,37015.0,240.0,0210001002,0101000110,42.0,13.0,6.0,0.0,0.0,5.0,0.0,1.0,0.0,3.0,0.0,8.0,0.0,1.0,1.0,0.0,10.0,7.0,4.0,4.0,0.0,0.0,30.0,4.0,0.0,0.0,0.0,0.0,40.0,11.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,6.0,0.0,14.0,2.0,0.0,0.0,0.0,12.0,7.0,5.0,5.0,4.0,0.0,30.0,15.0,2.0,0.0,1.0,0.0,dejer901,Ramon De Jesus,wintm901,Mike Winters,muchm901,Mike Muchlinski,wegnm901,Mark Wegner,NaN,NaN,NaN,NaN,cashk001,Kevin Cash,banij001

game_log dataframe records every single game's information. Every column's introduction is described in the game_log_field.txt
There are many id's in this dataframe. These IDs can be used in linking tables.
linking columns: park_id.

In [7]:
print(park_codes.shape)
park_codes.head(10)

(252, 9)


,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN
5,ATL01,Atlanta-Fulton County Stadium,NaN,Atlanta,GA,04/12/1966,09/23/1996,NL,NaN
6,ATL02,Turner Field,NaN,Atlanta,GA,04/04/1997,10/02/2016,NL,NaN
7,ATL03,Suntrust Park,NaN,Atlanta,GA,04/14/2017,NaN,NL,NaN
8,BAL01,Madison Avenue Grounds,NaN,Baltimore,MD,07/08/1871,07/08/1871,NaN,WS3
9,BAL02,Newington Park,NaN,Baltimore,MD,04/22/1872,09/30/1882,NaN,BL1:1872-74; BL4:1873; BL2: 1882


Park_codes dataframe describes the parking information for each game. Type of data includes name, date, location.

In [8]:
print(person_codes.shape)
person_codes.head()

(20494, 7)


,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


Person_codes dataframe discribes each player in the league including name, date of debut, mgr_debut, coach_debut and ump_debut.

In [9]:
print(team_codes.shape)
team_codes.head()

(150, 8)


,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


team_codes dataframe describes each baseball team. including starting and ending date. City of origin.

From Wiki the definition of the defensive position number: There are nine fielding positions in baseball. Each position conventionally has an associated number, which is used to score putouts: 1 (pitcher), 2 (catcher), 3 (first baseman), 4 (second baseman), 5 (third baseman), 6 (shortstop), 7 (left fielder), 8 (center fielder), and 9 (right fielder).

## Pandas SQL read

In [10]:
import sqlite3

db = 'mlb.db'

# run_query function takes a SQL query as an argument and returns a pandas dataframe of that query
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q,conn)

# run_command function takes a SQL command as an argument and executes it using the sqlite module
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)
        
# show_tables function calls the run_query() function to return a list of all tables and views in the database
def show_tables():
    q = '''
        SELECT
            name,
            type
        FROM sqlite_master
        WHERE type IN ('table','view')
        '''
    return run_query(q)

In [20]:
tables = {
    "game_log": game_log,
    "person_codes": person_codes,
    "team_codes": team_codes,
    "park_codes": park_codes
}

with sqlite3.connect(db) as conn:    
    for name, data in tables.items():
        conn.execute("DROP TABLE IF EXISTS {};".format(name))
        data.to_sql(name,conn,index=False)

In [21]:
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game,table
6,person_codes,table
7,team_codes,table
8,park_codes,table
9,game_log,table


In [22]:
c1 = '''
    ALTER TABLE game_log
    ADD COLUMN game_id TEXT;
    '''
try:
    run_command(c1)
except:
    pass

c2 = '''
    UPDATE game_log
    SET game_id = h_name || date || number_of_game
    /* WHERE prevents this if it has already been done */
    WHERE game_id IS NULL;
    '''
run_command(c2)

q = '''
    SELECT 
        game_id,
        h_name,
        date,
        number_of_game
    FROM game_log
    LIMIT 5;
    '''
run_query(q)

,game_id,h_name,date,number_of_game
0,FW1187105040,FW1,18710504,0
1,WS3187105050,WS3,18710505,0
2,RC1187105060,RC1,18710506,0
3,CH1187105080,CH1,18710508,0
4,TRO187105090,TRO,18710509,0


## Data Normalization:

The following are the opportunities for normalization of our data:
- In person_codes, all the debut dates will be able to reporoduced using game log data.
- In team_codes, the start, end and sequence columns will be able to be reproduced using game log data.
- In park_codes, the start and end years will be able to reproduced using game log data. While technoically the state is an attribute of the city, we might not want to have an incomplete city/state table so we will leave this in.
- There are lots of places in game log where we have a player ID followed by the players name. We will be able to remove this and use the name data in person_codes.
- In game_log, all offensive and defensive stats are repeated for the home team and the visting team. We could break these out and have a table that lists each game twice, one for each team, and cut out this column repetition.
- Similarly, in game_log, we have a listing for 9 players on each team with their positions - we can remove these and have one table that tracks player appearances and their positions.
- We can do a similar thing with the umpires from game_log. Instead of listing all four positons as columns, we can put the umpires either in the their table or make one table for players, umpires and managers.
- We have several awards in game_log like winning pitcher and losing pitcher. We can either break these out into their own table, have a table for awards, or combine the awards in with general appearances like the players and umpires.

## Planning a Normalized Schema:

![schema](C:/Users/andre/Desktop/schema-screenshot.png)

## Creating Tables Without Foreign Keys:

### Table: person

In [23]:
c1 = """
CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
"""

c2 = """
INSERT OR IGNORE INTO person
SELECT
    id,
    first,
    last
FROM person_codes;
"""

q = """
SELECT * FROM person
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


### Table: park

In [24]:
c1 = '''
    CREATE TABLE IF NOT EXISTS park (
        park_id TEXT PRIMARY KEY,
        name TEXT,
        nickname TEXT,
        city TEXT,
        state TEXT,
        notes TEXT
    );
    '''

c2 = '''
    INSERT OR IGNORE INTO park
    SELECT
        park_id,
        name,
        aka,
        city,
        state,
        notes
    FROM park_codes;
    '''

q = '''
    SELECT * FROM park
    LIMIT 5;
    '''

run_command(c1)
run_command(c2)
run_query(q)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


### Table: league

In [26]:
c1 = """
CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT
);
"""

c2 = """
INSERT OR IGNORE INTO league
VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association")
;
"""

q = """
SELECT * FROM league
"""

run_command(c1)
run_command(c2)
run_query(q)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


### Table: appearance_type

In [28]:
c = 'DROP TABLE IF EXISTS appearance_type'
run_command(c)

c1 = '''
    CREATE TABLE IF NOT EXISTS appearance_type (
        appearance_type_id TEXT PRIMARY KEY,
        name TEXT,
        category TEXT
    );
    '''
run_command(c1)

appearance_type = pd.read_csv('appearance_type.csv')

with sqlite3.connect(db) as conn:    
    appearance_type.to_sql('appearance_type',conn,index=False,if_exists='append')

q = 'SELECT * FROM appearance_type LIMIT 5'

run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


## Team and Game Tables:

In [29]:
c1 = '''
    CREATE TABLE IF NOT EXISTS team (
        team_id TEXT PRIMARY KEY,
        league_id TEXT,
        city TEXT,
        nickname TEXT,
        franch_id,
        FOREIGN KEY (league_id) REFERENCES leauge(league_id)
    );
    '''

c2 = '''
    INSERT OR IGNORE INTO team
        SELECT
            team_id,
            league,
            city,
            nickname,
            franch_id
        FROM team_codes
    ;
    '''

q = 'SELECT * FROM team LIMIT 5'


run_command(c1)
run_command(c2)
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [30]:
c1 = '''
    CREATE TABLE IF NOT EXISTS game (
        game_id TEXT PRIMARY KEY,
        date TEXT,
        number_of_game INTEGER,
        park_id TEXT,
        length_outs INTEGER,
        day BOOLEAN,
        completion TEXT,
        forefeit TEXT,
        protest TEXT,
        attendance INTEGER,
        length_minutes INTEGER,
        additional_info TEXT,
        acquisition_info TEXT,
        FOREIGN KEY (park_id) REFERENCES park(park_id)
    );
    '''

c2 = '''
    INSERT OR IGNORE INTO game
        SELECT
            game_id,
            date,
            number_of_game,
            park_id,
            length_outs,
            CASE
                WHEN day_night = "D" THEN 1
                WHEN day_night = "N" THEN 0
                ELSE NULL
                END
                AS day,
            completion,
            forefeit,
            protest,
            attendance,
            length_minutes,
            additional_info,
            acquisition_info
        FROM game_log;
    '''

q = 'SELECT * FROM game LIMIT 5'

run_command(c1)
run_command(c2)
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


## Table: team_appearance:

In [41]:
c1 = '''
    CREATE TABLE IF NOT EXISTS team_appearance (
        team_id TEXT,
        game_id TEXT,
        home BOOLEAN,
        league_id TEXT,
        score INTEGER,
        line_score TEXT,
        at_bats INTEGER,
        hits INTEGER,
        doubles INTEGER,
        triples INTEGER,
        homeruns INTEGER,
        rbi INTEGER,
        sacrifice_hits INTEGER,
        sacrifice_flies INTEGER,
        hit_by_pitch INTEGER,
        walks INTEGER,
        intentional_walks INTEGER,
        strikeouts INTEGER,
        stolen_bases INTEGER,
        caught_stealing INTEGER,
        grounded_into_double INTEGER,
        first_catcher_interferemce INTEGER,
        left_on_base INTEGER,
        pitchers_used INTEGER,
        individual_earned_runs INTEGER,
        team_earned_runs INTEGER,
        wild_pitches INTEGER,
        balks INTEGER,
        putouts INTEGER,
        assists INTEGER,
        errors INTEGER,
        passed_balls INTEGER,
        double_plays INTEGER,
        triple_plays INTEGER,
        PRIMARY KEY (team_id, game_id),
        FOREIGN KEY (team_id) REFERENCES team(team_id),
        FOREIGN KEY (game_id) REFERENCES game(game_id),
        FOREIGN KEY (league_id) REFERENCES league(league_id)
    );'''

c2 = '''
    INSERT OR IGNORE INTO team_appearance
        SELECT
            h_name,
            game_id,
            1 AS home,
            h_league,
            h_score,
            h_line_score,
            h_at_bats,
            h_hits,
            h_doubles,
            h_triples,
            h_homeruns,
            h_rbi,
            h_sacrifice_hits,
            h_sacrifice_flies,
            h_hit_by_pitch,
            h_walks,
            h_intentional_walks,
            h_strikeouts,
            h_stolen_bases,
            h_caught_stealing,
            h_grounded_into_double,
            h_first_catcher_interference,
            h_left_on_base,
            h_pitchers_used,
            h_individual_earned_runs,
            h_team_earned_runs,
            h_wild_pitches,
            h_balks,
            h_putouts,
            h_assists,
            h_errors,
            h_passed_balls,
            h_double_plays,
            h_triple_plays
        FROM game_log
        
    UNION
        SELECT
            v_name,
            game_id,
            0 AS home,
            v_league,
            v_score,
            v_line_score,
            v_at_bats,
            v_hits,
            v_doubles,
            v_triples,
            v_homeruns,
            v_rbi,
            v_sacrifice_hits,
            v_sacrifice_flies,
            v_hit_by_pitch,
            v_walks,
            v_intentional_walks,
            v_strikeouts,
            v_stolen_bases,
            v_caught_stealing,
            v_grounded_into_double,
            v_first_catcher_interference,
            v_left_on_base,
            v_pitchers_used,
            v_individual_earned_runs,
            v_team_earned_runs,
            v_wild_pitches,
            v_balks,
            v_putouts,
            v_assists,
            v_errors,
            v_passed_balls,
            v_double_plays,
            v_triple_plays
        FROM game_log;
        '''
q = '''SELECT * FROM team_appearance
        WHERE game_id = (
                        SELECT MIN(game_id) FROM game
                        )
            OR game_id = (
                        SELECT MAX(game_id) FROM game
                        )
        ORDER BY game_id, home;
        '''
q1 = 'SELECT * FROM team_appearance ORDER BY game_id LIMIT 50'
run_command(c1)
run_command(c2)
run_query(q)
run_query(q1)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interferemce,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SLU,ALT188404300,0,UA,15,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALT,ALT188405020,1,UA,3,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SLU,ALT188405020,0,UA,16,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALT,ALT188405030,1,UA,5,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SLU,ALT188405030,0,UA,14,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ALT,ALT188405050,1,UA,2,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SLU,ALT188405050,0,UA,12,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ALT,ALT188405100,1,UA,9,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BSU,ALT188405100,0,UA,4,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Table: person_appearance:

In [53]:
c0 = "DROP TABLE IF EXISTS person_appearance"

run_command(c0)

c1 = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c2 = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

run_command(c1)
run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [54]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_person_appearance FROM person_appearance"))

q = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q)


   games_game
0      171907
   games_person_appearance
0                   171907


,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1646114,steab101,None,WSU188409250,UHP,Home Plate,umpire
1,1646116,murnt101,BSU,WSU188409250,MM,Manager,manager
2,1646115,crane101,BSU,WSU188409250,PSP,Starting Pitcher,pitcher
3,1646118,scanm801,WSU,WSU188409250,MM,Manager,manager
4,1646117,dailh101,WSU,WSU188409250,PSP,Starting Pitcher,pitcher


## Droping unused tables:

In [55]:
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,team,table
4,game,table
5,person_codes,table
6,team_codes,table
7,park_codes,table
8,game_log,table
9,appearance_type,table


In [56]:
tables = [
    "game_log",
    "park_codes",
    "team_codes",
    "person_codes"
]

for t in tables:
    c = '''
    DROP TABLE {}
    '''.format(t)
    
    run_command(c)

show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,team,table
4,game,table
5,appearance_type,table
6,team_appearance,table
7,person_appearance,table
